# Sessió 2: Pytorch & Classificació

## **NOM**: ####

## **NIU**: ####

En aquesta sessió farem una petita introducció dels components i el procés que s'ha de realitzar al entrenar un model en **PyTorch**.

* Durant la classe, implementarem un regresor logistic binari one-vs-rest.
* A casa
 * Adaptar aquest codi per a que simuli un regressor logistic binari one-vs-one depenent del vostre **NIU**
 * Visualitzar el model
 * Aplicar regularització i veure diferencies.
 * Crear classificador multicategoria a partir de molts classificadors independents one-vs-rest.

Treballarem sobre la base de dades MNIST. Més avall teniu el codi per la seva descarrega.

### Imports

In [1]:
import argparse
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

### Parameters

En principi son parametres força estandard. No haurieu de per què tocar-los gaire si no sabeu el que volen dir.

In [2]:
batch_size = 100        # number of samples during training
test_batch_size = 1000  # number of samples for test 
epochs = 5              # number of epochs to train (default: 14)
lr = 0.01               # learning rate (default: 1.0)
gamma = 0.7             # Learning rate step gamma (default: 0.7)

no_cuda = True          # disables CUDA training
dry_run = False         # quickly check a single pass
seed = 1                # random seed (default: 1)
log_interval = 50       # how many batches to wait before logging training status
save_model = False      # For Saving the current Model


# Check if cuda is available
use_cuda = not no_cuda and torch.cuda.is_available()
print(f"USING CUDA: {use_cuda}")
torch.manual_seed(seed)

# define the device where to compute (cpu or gpu)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

USING CUDA: False


### Definició del Model

Creem un model lineal amb 784 entrades (la mida de la imatge (28x28)) i 1 sortida (Classificació binaria, és o no és)

In [3]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.w = torch.nn.Linear(784, 1)      # creem un model amb 784 entrades i 1 sortida

    def forward(self, x):                    # x:  bsx28x28
        x = torch.flatten(x, 1)              # flatten converteix una matriu en un array  x-> 28x28 -> 784x1
        x = self.w(x)                        # aplica els pesos sum(w*x) ->  784x1 -> 1
        x = torch.sigmoid(x)            # aplica la sigmoid  <- SVM la fa servir??
        return torch.flatten(x, 0)      # return bs probs


### Definició de la Loss

In [4]:
class LogisticLoss(torch.nn.modules.Module):
    def __init__(self):
        super(LogisticLoss, self).__init__()

    def forward(self, outputs, labels):
        batch_size = outputs.size()[0]
        outputs = (outputs * 2) - 1
        labels = (labels * 2) - 1  # labels -> 1 or -1
        return torch.sum(torch.log(1 + torch.exp(-(outputs.t() * labels)))) / batch_size

class HingeLoss(torch.nn.modules.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, outputs, labels):
        batch_size = outputs.size()[0]
        outputs = (outputs * 2) - 1
        labels = (labels * 2) - 1   # labels -> 1 or -1
        return torch.sum(torch.clamp(1 - outputs.t() * labels, min=0)) / batch_size # modificar el 1 per la loss corresponent


Recordeu la formula de la hinge loss: $ \frac{1}{n}\sum max(0, 1 - t * y) $

### Funcions auxiliars

In [5]:
def visualize_model(model):
    # Apartat B. Mostrar pesos del model
    plt.figure(0)
    #extraiem els pesos del model i els reformatajem en format imatge.
    plt.imshow(model.w.weight.detach().numpy().reshape(28, 28), interpolation='nearest', cmap=plt.cm.RdBu)
    plt.show()
    
def visualize_confusion_matrix(y_pred, y_real):
    #mostra la matriu de confusió
    cm = confusion_matrix(y_real, y_pred)
    plt.subplots(figsize=(10, 6))
    sns.heatmap(cm, annot = True, fmt = 'g')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()


def convert_dataset_to_binary_one_vs_one(dataset, positive_class, negative_class):
    # NIU: 12345678
    # One-vs-One
    # TODO. Apartat A. Aqui pots fer servir la funcio subset per tal reduir el nombre d'exemples
    # funcions que podeu fer servir:
    # torch.nonzero(X, as_tuple=True)[0]
    # torch.logical_or(X1,X2)
    # dataset = torch.utils.data.Subset(dataset, index_of_interest)

    print(f"SIZE of dataset {len(dataset)}")
    return dataset
    

def convert_dataset_to_binary_one_vs_rest(dataset, positive_class):
    # One-vs-Rest    
    dataset.targets[dataset.targets != positive_class] = -1.0
    dataset.targets[dataset.targets == positive_class] = 1.0
    dataset.targets[dataset.targets == -1.0] = 0.0
        
    print(f"SIZE of dataset {len(dataset)}")
    return dataset

### Train loop (1 epoca)

In [6]:
def train(model, device, train_loader, optimizer, criterion, epoch, log_interval):
    #Ens posem en mode entrenament.
    model.train()
    #iterem les dades en batches (o lots)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # en el cas de treballar en cuda o gpu, ara els hi posariem
        optimizer.zero_grad()  # posem el gradient a zero (important) ja que sino, s'aniria acumulant a cada iteració
        output = model(data)   # fem un predict del model amb les dades actuals
        loss = criterion(output.view_as(target), target.type_as(output))  # calculem el error obtingut
        loss.backward()             # li diem que calculi el gradient
        optimizer.step()            # actualitza els pesos amb el gradient calculat
        if batch_idx % log_interval == 0:  # cada log_interval loops mostrem informació
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            #visualize_model(model)


### Test Loop

In [7]:
def test(model, device, test_loader, criterion, show_confusion_matrix=False):
    model.eval()     # ens posem en mode evaluació
    test_loss = 0    # variables per acumular resultats
    correct = 0
    all_preds = [] 
    all_outputs = [] 
    all_targets = []
    with torch.no_grad():  # li estem dient que estem en evaluacio, i que necesitem calcular el gradient aqui dins 
        for data, target in test_loader:   # per cada batch de test
            data, target = data.to(device), target.to(device)  # posar-ho a la cpu o a la gpu
            output = model(data)    # fer la prediccio
            test_loss += criterion(output.view_as(target), target.type_as(output)).item() * data.shape[0]  # acumulem error
            pred = (output > 0.5) * 1  # per comprobar la prediccio escollim el threshold a 0.5
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred)
            all_targets.extend(target)
            all_outputs.extend(output)

    test_loss /= len(test_loader.dataset) # fem la mitja de l'error del dataset
    if show_confusion_matrix:
        visualize_confusion_matrix(all_targets, all_preds)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return all_outputs

### Preparar les dades d'entrenament

In [8]:
# Definim un pipeline que se li aplicarà a les dades del dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Aqui agafem les dades del propi torch. Si no les tenim, amb el parametre download ens el baixarem automaticament.
# Esculliu on voleu descarregar-vos les dades i intenteu de no anar-les replicant per totes les carpetes..
dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('./data', train=False, transform=transform)

# TODO. Apartat A. Aqui convertim el dataset en binari one-vs-rest.
#  Modifiqueu la funcio per adaptar el vostre niu
# convert multiclass dataset into binary classes
dataset1 = convert_dataset_to_binary_one_vs_rest(dataset1, 0)
dataset2 = convert_dataset_to_binary_one_vs_rest(dataset2, 0)

#dataset1 = convert_dataset_to_binary_one_vs_one(dataset1, 0, 1)
#dataset2 = convert_dataset_to_binary_one_vs_one(dataset2, 0, 1)

train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)



RuntimeError: ./data/MNIST/processed/training.pt is a zip archive (did you mean to use torch.jit.load()?)

### Instanciació i entrenament

In [ ]:
model = Model().to(device)  # instanciació del model
optimizer = torch.optim.SGD(model.parameters(), lr=lr)   # optimitzador (Apartat C. Aqui podem definir el weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)  # permet reduir el Learning rate

# criterion = LogisticLoss()
# criterion = HingeLoss()
criterion = torch.nn.BCELoss(reduction='mean')

for epoch in range(1, epochs + 1):
    # A cada epoca, fem un entrenament del model (es veu un cop cada exemple)
    print("LEARNING RATE USED: {}".format(scheduler.get_last_lr()))
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval)
    test(model, device, test_loader, criterion)
    scheduler.step()  # reduim el Learning Rate segons el scheduler StepLR que fa servir gamma

if save_model:
    torch.save(model.state_dict(), "mnist.pt")


### Multiples models

In [ ]:
#Apartat D
epochs = 1  # per anar més rapid... només fem una epoca per número..

output = np.zeros((10000, 10)) # matriu on guardarem els resultats (10000 samples x 10 categories)

for i_categoria in range(10):
    print(f"CATEGORIA {i_categoria}")
    # agafem tot el dataset
    dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST('./data', train=False, transform=transform)

    # seleccionem i modifiquem els labels per convertir el dataset a classificació binaria
    dataset1 = convert_dataset_to_binary_one_vs_rest(dataset1, i_categoria)
    dataset2 = convert_dataset_to_binary_one_vs_rest(dataset2, i_categoria)

    train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Model().to(device)  # instanciar el model
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)  # instanciar el optimitzador
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)

    criterion = torch.nn.BCELoss(reduction='mean')  # definir la loss

    for epoch in range(1, epochs + 1):
        # A cada epoca, fem un entrenament del model (es veu un cop cada exemple)
        print("LEARNING RATE USED: {}".format(scheduler.get_last_lr()))
        train(model, device, train_loader, optimizer, criterion, epoch, log_interval)
        if epoch==epochs:  # a la ultima iteració del training, fem la predicció
            i_output = test(model, device, test_loader, criterion, show_confusion_matrix=False)
            output[:, i_categoria] = np.asarray(i_output)
            
        scheduler.step()  # reduim el Learning Rate segons el scheduler StepLR que fa servir gamma
    
    if save_model:
        torch.save(model.state_dict(), f"mnist_{i_categoria}.pt")


In [ ]:
# Apartat D
# a output, tenim totes les prediccions, per cada sample, la predicció de cada categoria. hem de quedar-nos amb la categoria més probable
# agafem els labels originals de test
# visualitzar matriu de confusio (per extreurels de dataset: dataset2.targets.numpy())
# calcular el accuracy final multicategoria
multiclass_accuracy = 0
print(f"FINAL ACCURACY: {multiclass_accuracy}")

## Entrega

* A. Seleccioneu les categories segons el vostre NIU. Feu One-VS-One. Fer un entrenament i mostrar l'accuracy i la matriu de confusió. **(3pts)**

* B. Visualitzar i comentar els pesos w en format imatge. Fer servir la funcion visualize_model(model). Potser a diferents punts del entrenament? **(3pts)**

* C. Què passa si regularitzeu els pesos. Podrieu sumarli el cost a la loss, o bé, utilitzar el ``weight_decay`` en el Optimitzador, que aplica un L2 penalty. Proveu varis valors. [Explicació](https://towardsdatascience.com/this-thing-called-weight-decay-a7cd4bcfccab) **(2pts)**

* D. Fer 10 classificadors. Cada un one-vs-rest. (ie. 0vsRest, 1vsRest..). Aplicar els 10 models al test. Agafar la categoria més probable de les 10 i mostrar la matriu de confusió resultant. Quin accuracy aconseguiu? **(2pts)**
